# Análisis exploratorio y limpieza de datos

In [1]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import glob, os
import datetime
import matplotlib.pyplot as plt

## Importando los csv's

In [2]:
## Importando los datos para PM10
path = r'C:\Users\luisf\Documents\Python Projects\datamex1019\air-quality-cdmx\data_aircdmx'                     # use your path
all_files = glob.glob(os.path.join(path, "*PM10.xls"))     # advisable to use os.path.join as this makes concatenation OS independent
all_files

['C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2000PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2001PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2002PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2003PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2004PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2005PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2006PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2007PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data_aircdmx\\2008PM10.xls',
 'C:\\Users\\luisf\\Document

In [3]:
df_from_each_file = (pd.read_excel(f) for f in all_files)
df_from_each_file
concatenated_df=pd.concat([i.set_index(['FECHA']+['HORA']) for i in df_from_each_file]).reset_index()

In [ ]:
concatenated_df.shape

In [ ]:
concatenated_df.columns

In [4]:
#Filtrando por las alcaldías dentro del alcance
df=concatenated_df[["FECHA","HORA","BJU","HGM","MGH"]]

In [ ]:
df.dtypes

In [5]:
df.HORA[df.HORA==24] = 00
df.FECHA=df.FECHA.apply(str)
df.FECHA=df.FECHA.apply(lambda x: x.replace(' 00:00:00',''))
df['DATETIME']=df.FECHA + ' ' + df.HORA.apply(str)+':00:00'
#df.DATETIME=df.DATETIME.apply(lambda x: x.replace(' 00:00:00',''))
df.DATETIME=pd.to_datetime(df.DATETIME)
df.DATETIME.head()

0   2000-01-01 01:00:00
1   2000-01-01 02:00:00
2   2000-01-01 03:00:00
3   2000-01-01 04:00:00
4   2000-01-01 05:00:00
Name: DATETIME, dtype: datetime64[ns]

In [ ]:
df.FECHA.head()

In [ ]:
df.head()

## ¿Cómo le haré para manejar los NaN's y los valores -99.0 (que son lo mismo)?
- Primero, dividiré el data set por cada estación

In [6]:
#df[["FECHA","HORA","BJU","HGM","MGH"]]
df_benito=df[["DATETIME","FECHA","HORA","BJU"]]
df_cuauh=df[["DATETIME","FECHA","HORA","HGM"]]
df_miguel=df[["DATETIME","FECHA","HORA","MGH"]]

## Limpieza para Benito Juárez

In [7]:
nulls_benito = {}
for i in range(2000,2020):
    nulls_benito[i]=(df_benito.BJU[(df_benito.FECHA >= '{}-01-01' .format(i)) & (df_benito.FECHA <= '{}-12-31' .format(i))].isnull().sum()[0])

In [8]:
nulls_benito #Esto quiere decir que para los primeros 14 años de este milenio, no se midió nada en esta estación

{2000: 0,
 2001: 0,
 2002: 0,
 2003: 0,
 2004: 0,
 2005: 0,
 2006: 0,
 2007: 0,
 2008: 0,
 2009: 0,
 2010: 0,
 2011: 0,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

In [19]:
df_benito_v2=df_benito[df_benito.FECHA >= '2015-01-01']
df_benito_v2.shape

(42360, 4)

In [20]:
df_benito_v2.BJU.value_counts().head() # 10 mil putos -99.0 

-99.0    10581
 25.0      707
 24.0      679
 23.0      673
 26.0      670
Name: BJU, dtype: int64

In [21]:
df_benito_v2['AÑO']=df_benito_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_benito_v2['MES']=df_benito_v2.FECHA.apply(lambda x: str(x).split('-')[1])
df_benito_v2.AÑO=pd.to_numeric(df_benito_v2.AÑO)
df_benito_v2.MES=pd.to_numeric(df_benito_v2.MES)

In [22]:
df_benito_v2.head()

,DATETIME,FECHA,HORA,BJU,AÑO,MES
131496,2015-01-01 01:00:00,2015-01-01,1,-99.0,2015,1
131497,2015-01-01 02:00:00,2015-01-01,2,-99.0,2015,1
131498,2015-01-01 03:00:00,2015-01-01,3,-99.0,2015,1
131499,2015-01-01 04:00:00,2015-01-01,4,-99.0,2015,1
131500,2015-01-01 05:00:00,2015-01-01,5,-99.0,2015,1


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [16]:
df_benito_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2015,1,744
1,2015,2,672
2,2015,3,744
3,2015,4,720
4,2015,5,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

In [17]:
df_benito_v2[df_benito_v2.BJU == -99.0].groupby(["AÑO","MES"])['BJU'].value_counts().to_frame().head()

#Quitar los meses 01,02,03,04,05,06 y 07 del año 2015
#Quitar el 2019 por lo pronto, para la primera predicción.
#Llenar los demás valores con un promedio del año correspondiente

BJU
AÑO  MES BJU       
2015 1   -99.0  744
     2   -99.0  672
     3   -99.0  744
     4   -99.0  720
     5   -99.0  744

In [18]:
#Filtrando datos vaciós de los primeros 7 meses y nada del 2019
df_benito_v3=df_benito_v2[(df_benito_v2.FECHA >= '2015-08-01') & (df_benito_v2.FECHA <= '2018-12-31')]

In [19]:
means = []
for e in range(2015,2019):
    means.append(df_benito_v3.BJU[df_benito_v3.AÑO == e].mean())

df_benito_v3.BJU[(df_benito_v3.AÑO == 2015) & (df_benito_v3.BJU == -99.0)] = means[0]
df_benito_v3.BJU[(df_benito_v3.AÑO == 2016) & (df_benito_v3.BJU == -99.0)] = means[1]
df_benito_v3.BJU[(df_benito_v3.AÑO == 2017) & (df_benito_v3.BJU == -99.0)] = means[2]
df_benito_v3.BJU[(df_benito_v3.AÑO == 2018) & (df_benito_v3.BJU == -99.0)] = means[3]

In [20]:
df_benito_v3[df_benito_v3.BJU== -99.0] #Correcto

,DATETIME,FECHA,HORA,BJU,AÑO,MES


## Limpieza para Cuahtémoc

### Encontrar NaN's y -99.0

In [21]:
nulls_cuauh = {}
for i in range(2000,2020):
    nulls_cuauh[i]=(df_cuauh[(df_cuauh.FECHA >= '{}-01-01' .format(i)) & (df_cuauh.FECHA <= '{}-12-31' .format(i))].isnull().sum()[2])

In [22]:
nulls_cuauh #Medición a partir del 2012

{2000: 0,
 2001: 0,
 2002: 0,
 2003: 0,
 2004: 0,
 2005: 0,
 2006: 0,
 2007: 0,
 2008: 0,
 2009: 0,
 2010: 0,
 2011: 0,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

In [7]:
df_cuauh_v2=df_cuauh[df_cuauh.FECHA >= '2012-01-01']
df_cuauh_v2.shape

(68664, 4)

In [8]:
df_cuauh_v2['AÑO']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_cuauh_v2['MES']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[1])
df_cuauh_v2.AÑO=pd.to_numeric(df_cuauh_v2.AÑO)
df_cuauh_v2.MES=pd.to_numeric(df_cuauh_v2.MES)

In [25]:
df_cuauh_v2.head()

,DATETIME,FECHA,HORA,HGM,AÑO,MES
105192,2012-01-01 01:00:00,2012-01-01,1,-99.0,2012,1
105193,2012-01-01 02:00:00,2012-01-01,2,-99.0,2012,1
105194,2012-01-01 03:00:00,2012-01-01,3,-99.0,2012,1
105195,2012-01-01 04:00:00,2012-01-01,4,-99.0,2012,1
105196,2012-01-01 05:00:00,2012-01-01,5,-99.0,2012,1


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [9]:
df_cuauh_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2012,1,744
1,2012,2,696
2,2012,3,744
3,2012,4,720
4,2012,5,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

In [10]:
df_cuauh_v2[df_cuauh_v2.HGM == -99.0].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame().head()

HGM
AÑO  MES HGM       
2012 1   -99.0  744
     2   -99.0  487
     3   -99.0   23
     4   -99.0   42
     5   -99.0  193

In [11]:
df_cuauh_v2[(df_cuauh_v2.HGM == -99.0) & (df_cuauh_v2.AÑO.isin(["2014","2015","2016","2017"]))].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame().head()

#En conclusión:
#Eliminaré el mes 07 y 11 del año 2014. 09 y 10 del 2015. 01 y 07 del 2017
#Eliminar mes 01, 02 del 2012. 02 y 09 del 2018
#Llenar con con mean del año los otros -99.0

HGM
AÑO  MES HGM       
2014 1   -99.0   10
     2   -99.0   39
     3   -99.0   20
     4   -99.0   11
     5   -99.0  130

In [12]:
#Eliminaré el mes 07 y 11 del año 2014. 09 y 10 del 2015. 01 y 07 del 2017
#Eliminar mes 01, 02 del 2012. 02 y 09 del 2018
df_cuauh_v2[(df_cuauh_v2.AÑO == 2014) & (df_cuauh_v2.MES.isin([7,11]))]
df_cuauh_v3=df_cuauh_v2.copy()

In [13]:
def drop_99_rows(dataset,año,mes1,mes2):
    for e in dataset[(dataset.AÑO == año) & (dataset.MES.isin([mes1,mes2]))].index:
        dataset = dataset.drop(e)
    return dataset


df_cuauh_v3=drop_99_rows(df_cuauh_v2,2014,7,11)
df_cuauh_v3=drop_99_rows(df_cuauh_v3,2015,9,10)
df_cuauh_v3=drop_99_rows(df_cuauh_v3,2017,1,7)
df_cuauh_v3=drop_99_rows(df_cuauh_v3,2012,1,2)
df_cuauh_v3=drop_99_rows(df_cuauh_v3,2018,2,9)

#2014
#for e in df_cuauh_v2[(df_cuauh_v2.AÑO == 2014) & (df_cuauh_v2.MES.isin([7,11]))].index:
#    df_cuauh_v3 = df_cuauh_v3.drop(e)

In [14]:
df_cuauh_v3[(df_cuauh_v3.HGM == -99.0) & (df_cuauh_v3.AÑO.isin([2014,2015,2016,2017,2018,2012]))].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame().head()

HGM
AÑO  MES HGM       
2012 3   -99.0   23
     4   -99.0   42
     5   -99.0  193
     6   -99.0  123
     7   -99.0   82

In [32]:
df_cuauh_v3[df_cuauh_v3.HGM == -99.0].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame().head()

HGM
AÑO  MES HGM       
2012 3   -99.0   23
     4   -99.0   42
     5   -99.0  193
     6   -99.0  123
     7   -99.0   82

In [15]:
means = []
for e in range(2012,2019):
    means.append(df_cuauh_v3.HGM[df_cuauh_v3.AÑO == e].mean())
#means
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2012) & (df_cuauh_v3.HGM == -99.0)] = means[0]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2013) & (df_cuauh_v3.HGM == -99.0)] = means[1]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2014) & (df_cuauh_v3.HGM == -99.0)] = means[2]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2015) & (df_cuauh_v3.HGM == -99.0)] = means[3]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2016) & (df_cuauh_v3.HGM == -99.0)] = means[4]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2017) & (df_cuauh_v3.HGM == -99.0)] = means[5]
df_cuauh_v3.HGM[(df_cuauh_v3.AÑO == 2018) & (df_cuauh_v3.HGM == -99.0)] = means[6]

In [16]:

df_cuauh_v3[df_cuauh_v3.HGM== -99.0] #Correcto

,DATETIME,FECHA,HORA,HGM,AÑO,MES
166620,2019-01-03 13:00:00,2019-01-03,13,-99.0,2019,1
166621,2019-01-03 14:00:00,2019-01-03,14,-99.0,2019,1
166622,2019-01-03 15:00:00,2019-01-03,15,-99.0,2019,1
166623,2019-01-03 16:00:00,2019-01-03,16,-99.0,2019,1
166624,2019-01-03 17:00:00,2019-01-03,17,-99.0,2019,1
166920,2019-01-16 01:00:00,2019-01-16,1,-99.0,2019,1
166921,2019-01-16 02:00:00,2019-01-16,2,-99.0,2019,1
166922,2019-01-16 03:00:00,2019-01-16,3,-99.0,2019,1
166923,2019-01-16 04:00:00,2019-01-16,4,-99.0,2019,1
166924,2019-01-16 05:00:00,2019-01-16,5,-99.0,2019,1


## Limpieza para Miguel Hidalgo

### Encontrar NaN's y -99.0

In [35]:
nulls_miguel = {}
for i in range(2000,2020):
    nulls_miguel[i]=(df_miguel[(df_miguel.FECHA >= '{}-01-01' .format(i)) & (df_miguel.FECHA <= '{}-12-31' .format(i))].isnull().sum()[2])

Las mediciones de esta estación empiezan a partir del 2015

In [36]:
nulls_miguel #Datos a partir del 2015

{2000: 0,
 2001: 0,
 2002: 0,
 2003: 0,
 2004: 0,
 2005: 0,
 2006: 0,
 2007: 0,
 2008: 0,
 2009: 0,
 2010: 0,
 2011: 0,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

Recortar el dataframe

In [37]:
df_miguel_v2=df_miguel[df_miguel.FECHA >= '2015-01-01']
df_miguel_v2.shape

(42360, 4)

In [38]:
df_miguel_v2.MGH.value_counts().head() # 6mil -99.0

-99.0    6048
 25.0     932
 27.0     896
 26.0     887
 23.0     874
Name: MGH, dtype: int64

Agregando columna AÑO y MES para hacer un groupby posterior, y así, encontrar los valores -99.0 para cada mes

In [39]:
df_miguel_v2['AÑO']=df_miguel_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_miguel_v2['MES']=df_miguel_v2.FECHA.apply(lambda x: str(x).split('-')[1])
df_miguel_v2.AÑO=pd.to_numeric(df_miguel_v2.AÑO)
df_miguel_v2.MES=pd.to_numeric(df_miguel_v2.MES)

In [40]:
df_miguel_v2.head()

,DATETIME,FECHA,HORA,MGH,AÑO,MES
131496,2015-01-01 01:00:00,2015-01-01,1,165.0,2015,1
131497,2015-01-01 02:00:00,2015-01-01,2,293.0,2015,1
131498,2015-01-01 03:00:00,2015-01-01,3,266.0,2015,1
131499,2015-01-01 04:00:00,2015-01-01,4,178.0,2015,1
131500,2015-01-01 05:00:00,2015-01-01,5,146.0,2015,1


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [41]:
df_miguel_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2015,1,744
1,2015,2,672
2,2015,3,744
3,2015,4,720
4,2015,5,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

Por ejemplo, para los últimos 5 meses del año 2019, practicamente no hay ningúna medición

In [57]:
df_miguel_v2[df_miguel_v2.MGH == -99.0].groupby(["AÑO","MES"])['MGH'].value_counts().to_frame()

MGH
AÑO  MES MGH       
2015 1   -99.0    7
     2   -99.0    6
     3   -99.0   34
     4   -99.0   58
     5   -99.0   83
     6   -99.0   38
     7   -99.0   14
     8   -99.0   12
     9   -99.0   29
     10  -99.0   31
     11  -99.0    5
     12  -99.0  144
2016 1   -99.0    5
     2   -99.0   10
     3   -99.0   21
     4   -99.0    4
     5   -99.0   11
     6   -99.0   23
     7   -99.0    9
     8   -99.0   12
     9   -99.0    5
     10  -99.0    6
     11  -99.0   11
     12  -99.0    7
2017 1   -99.0    6
     2   -99.0    6
     3   -99.0   11
     4   -99.0  266
     5   -99.0  272
     6   -99.0   19
     7   -99.0   29
     8   -99.0   46
     9   -99.0   38
     10  -99.0   46
     11  -99.0   20
     12  -99.0   33
2018 1   -99.0   10
     2   -99.0   32
     3   -99.0   11
     4   -99.0    4
     5   -99.0   13
     6   -99.0   19
     7   -99.0   15
     8   -99.0  286
     9   -99.0  153
     10  -99.0   25
     11  -99.0   63
     12  -99.0   18
2019 1   -99.0  128
     2   -99.0  207
     3   -99.0   26
     4   -99.0   40
     5   -99.0   26
     6   -99.0  643
     7   -99.0  744
     8   -99.0  744
     9   -99.0  720
     10  -99.0  744

In [43]:
df_miguel_v3=df_miguel_v2[df_miguel_v2.FECHA <= '2018-12-31']

In [44]:
# Llenar con medias
means = []
for e in range(2015,2019):
    means.append(df_miguel_v3.MGH[df_miguel_v3.AÑO == e].mean())

means
df_miguel_v3.MGH[(df_miguel_v3.AÑO == 2015) & (df_miguel_v3.MGH == -99.0)] = means[0]
df_miguel_v3.MGH[(df_miguel_v3.AÑO == 2016) & (df_miguel_v3.MGH == -99.0)] = means[1]
df_miguel_v3.MGH[(df_miguel_v3.AÑO == 2017) & (df_miguel_v3.MGH == -99.0)] = means[2]
df_miguel_v3.MGH[(df_miguel_v3.AÑO == 2018) & (df_miguel_v3.MGH == -99.0)] = means[3]

In [45]:
df_miguel_v3[df_miguel_v3.MGH== -99.0] #Correcto

,DATETIME,FECHA,HORA,MGH,AÑO,MES


In [46]:
df_miguel_v3.to_csv("data_clean/miguel_pm10_clean.csv")
df_cuauh_v3.to_csv("data_clean/cuauh_pm10_clean.csv")
df_benito_v3.to_csv("data_clean/benito_pm10_clean.csv")

In [50]:
df_miguel_v3.head()

,DATETIME,FECHA,HORA,MGH,AÑO,MES
131496,2015-01-01 01:00:00,2015-01-01,1,165.0,2015,1
131497,2015-01-01 02:00:00,2015-01-01,2,293.0,2015,1
131498,2015-01-01 03:00:00,2015-01-01,3,266.0,2015,1
131499,2015-01-01 04:00:00,2015-01-01,4,178.0,2015,1
131500,2015-01-01 05:00:00,2015-01-01,5,146.0,2015,1
